In [3]:
# TMDBML : TMDB 머신러닝 모델 만들기
import pandas as pd
import numpy as np

<pre>
# vote_average를 가중치를 부여
# vote = (v/(v+m))*R + (m/(v+m))*C
# v : 개별 영화에 투표한 횟수, m : 평점을 부여하기 위한 최소 투표 횟수
# R : 개별 영화에 대한 평균 평점(=vote_average), C : 전체 영화에 대한 평균 평점
</pre>

In [4]:
df = pd.read_csv("data/movie_data_final.csv")
df.head(1)  # title : str

,title,budget,genres,country,main_charactor,director,vote_average,vote_count
0,{616037: 'Thor: Love and Thunder'},250000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'iso_3166_1': 'US', 'name': 'United States o...",{3894: 'Christian Bale'},{55934: 'Taika Waititi'},6.772,1752


In [5]:
# m : 투표 횟수의 평균 * 60%
# C : 평균 평점 * 60 %
m = df.vote_average.mean() * 60 / 100
C = df.vote_count.mean() * 60 / 100
print("m : ", m)
print("C : ", C)

m :  3.927044210526323
C :  1494.659298245614


In [6]:
v = df.vote_count
R = df.vote_average
df['result'] = (v/(v+m))*R + (m/(v+m))*C
df.head(1)

,title,budget,genres,country,main_charactor,director,vote_average,vote_count,result
0,{616037: 'Thor: Love and Thunder'},250000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'iso_3166_1': 'US', 'name': 'United States o...",{3894: 'Christian Bale'},{55934: 'Taika Waititi'},6.772,1752,10.099587


In [7]:
import ast #형태가 리스트/딕셔너리인 문자열을 다시 리스트/딕셔너리 형태로 바꿀 수 있다.

In [8]:
# title: 이름, id/ genres: to_categorical/ country: to_categorical/ charactor: 이름, id/ director: 이름, id/ result 0 or 1
# numpy 배열 (5302,)
df_list = []
title, budget, genres, country, charactor, director, result = [], [], [], [], [], [], []
for idx in df.index:
    title.append([ast.literal_eval(df.title[idx])])
    budget.append([int(df.budget[idx])])
    genres.append(ast.literal_eval(df.genres[idx]))
    country.append(ast.literal_eval(df.country[idx]))
    charactor.append([ast.literal_eval(df.main_charactor[idx])])
    director.append([ast.literal_eval(df.director[idx])])
    result.append([1 if df.vote_average[idx] >= 0.5 else 0])
df_list.append([title, budget, genres, country, charactor, director, result])
df_arr = np.array(df_list)

C:\Users\ezen\AppData\Local\Temp\ipykernel_10000\1274597262.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df_arr = np.array(df_list)


In [9]:
# for title, charactor, director
class SeperateID_Name:
    def __init__(self, column):
        self.id = []
        self.name = []
        self.column = column

    def print(self):
        for col in self.column:
            for key, value in col[0].items():
                self.id.append(int(key))
                self.name.append(str(value))
        return self.id, self.name

In [10]:
# title : dictionary about title_ID & title_Name
sep_title = SeperateID_Name(title)
title_ID, title_Name = sep_title.print()
# charactor : dictionary about char_ID & char_Name
sep_charactor = SeperateID_Name(charactor)
char_ID, char_Name = sep_charactor.print()
# director : dictionary about dir_ID & dir_Name
sep_director = SeperateID_Name(director)
dir_ID, dir_Name = sep_director.print()

In [11]:
# country dictionry name to id 만들기
country_data = []
for cs in country:
    for c in cs:
        country_data.append(c['name'])
country_dict = {}
i = 0
for l in country_data:
    try:
        country_dict[l]  # 찾으면 패스 못찾으면 선언 및 아이디 부여
    except:
        country_dict[l] = i
        i += 1

In [12]:
# for genres, country
class SeperateID_Name_list:
    def __init__(self, column):
        self.ids, self.id = [], []
        self.names, self.name = [], []
        self.column = column

    def print(self):
        for cols in self.column:
            for col in cols:
                self.id.append(list(col.values())[0])
                self.name.append(list(col.values())[1])
            self.ids.append(self.id)
            self.names.append(self.name)
            self.id = []
            self.name = []
        return self.ids, self.names

In [13]:
# genres
sep_genres = SeperateID_Name_list(genres)
genre_ID, genre_Name = sep_genres.print()
# country
sep_country = SeperateID_Name_list(country)
country_ID, country_Name = sep_country.print()

In [14]:
# country_ID id 저장
country_I = []
country_ID = []
for names in country_Name:
    for name in names:
        country_I.append(country_dict[name])
    country_ID.append(country_I)
    country_I = []

- Id로 이루어진 데이터프레임(머신러닝용)

In [15]:
df.head(1)

,title,budget,genres,country,main_charactor,director,vote_average,vote_count,result
0,{616037: 'Thor: Love and Thunder'},250000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'iso_3166_1': 'US', 'name': 'United States o...",{3894: 'Christian Bale'},{55934: 'Taika Waititi'},6.772,1752,10.099587


In [46]:
# title	budget	genres	country	main_charactor	director	vote_average	vote_count	result
df_ID = pd.DataFrame({'title': title_ID, 'budget':df.budget, 'genres':genre_ID, 'country':country_ID, 'charactor': char_ID, 'director':dir_ID, 'vote_score':df.vote_average, 'vote_count':df.vote_count,
'vote_weight':df.result})

In [47]:
df_ID.head(1)

,title,budget,genres,country,charactor,director,vote_score,vote_count,vote_weight
0,616037,250000000,"[28, 12, 14]",[0],3894,55934,6.772,1752,10.099587


In [52]:
df_ID.to_csv('data/movie_data_ID.csv', index=False)

- Name으로 이루어진 데이터프레임(분석용)

In [49]:
# title	budget	genres	country	main_charactor	director	vote_average	vote_count	result
df_Name = pd.DataFrame({'title': title_Name, 'budget':df.budget, 'genres':genre_Name, 'country':country_Name, 'charactor': char_Name, 'director':dir_Name, 'vote_score':df.vote_average, 'vote_count':df.vote_count,
'vote_weight':df.result})

In [50]:
df_Name.head(1)

,title,budget,genres,country,charactor,director,vote_score,vote_count,vote_weight
0,Thor: Love and Thunder,250000000,"[Action, Adventure, Fantasy]",[United States of America],Christian Bale,Taika Waititi,6.772,1752,10.099587


In [53]:
df_Name.to_csv('data/movie_data_Name.csv', index=False)